In [68]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [69]:
!pip install requests_html

     |████████████████████████████████| 133kB 2.7MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [70]:
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen
import csv
import sys
import urllib
from urllib.error import HTTPError
from requests_html import HTMLSession
# import nest_asyncio

In [71]:
# nest_asyncio.apply()

session = HTMLSession()
off = 0
show = '&show=100'
sd_url = 'https://www.sciencedirect.com/search?qs=animal%20data%20star%20classification%20bag'
r = session.get(sd_url)
page_soup = soup(r.text, "html.parser")
result_count = 0
result_count = int(page_soup.find("span", {"class" : "search-body-results-text"}).text.strip().split()[0])
# print(result_count)
individual_urls = []
while(off<result_count):
  offset = "&offset=" + str(off)
  off += 100
  sd_url = 'https://www.sciencedirect.com/search?qs=animal%20data%20star%20classification%20bag' + show + offset
  r = session.get(sd_url)
  page_soup = soup(r.text, "html.parser")
  containers = page_soup.findAll("a", {"class" : "result-list-title-link"})
  url_prefix = "https://www.sciencedirect.com"

  for container in containers:
    container = container["href"];
    paper_url = url_prefix + container
    individual_urls.append(paper_url)
    print(paper_url)

https://www.sciencedirect.com/science/article/pii/S0950705117303106
https://www.sciencedirect.com/science/article/pii/S0263224118311692
https://www.sciencedirect.com/science/article/pii/S0092867416310078
https://www.sciencedirect.com/science/article/pii/B9780128213391000076
https://www.sciencedirect.com/science/article/pii/S1077314215000375
https://www.sciencedirect.com/science/article/pii/S152994301830247X
https://www.sciencedirect.com/science/article/pii/S0967064599001381
https://www.sciencedirect.com/science/article/pii/S092523121500675X
https://www.sciencedirect.com/science/article/pii/S0195666316308340
https://www.sciencedirect.com/science/article/pii/S0925231214003877
https://www.sciencedirect.com/science/article/pii/S0888754320311599
https://www.sciencedirect.com/science/article/pii/S0022098115300575
https://www.sciencedirect.com/science/article/pii/S095741741830558X
https://www.sciencedirect.com/science/article/pii/S2211124720300358
https://www.sciencedirect.com/science/article

In [72]:
print(len(individual_urls))

788


In [51]:
url = 'https://www.sciencedirect.com/sdfe/arp/cite'
suffix = '?pii=S0264410X18315482&format=text%2Fx-bibtex&withabstract=true'

In [53]:
!pip install pycurl 

     |████████████████████████████████| 225kB 2.8MB/s 
  Created wheel for pycurl: filename=pycurl-7.43.0.6-cp36-cp36m-linux_x86_64.whl size=284415 sha256=18e6b04b8bf584b4aa9fb548da26222ded20f5dfea618ae55c31988baa41dfdf
  Stored in directory: /root/.cache/pip/wheels/d9/a1/7b/2894883fd5c69a3ba684a49d255466d1e924b3cf58cedc1ade
Successfully built pycurl


In [97]:
import requests

headers = {
    'user-agent': 'My User Agent 1.0',
}
sd_url = 'https://www.sciencedirect.com/sdfe/arp/cite'
params = {
    'pii': None,
    'format': 'text/x-bibtex',
    'withabstract': 'true',
}

bibs_responses = []
for url in individual_urls:
  ind = len(url) - 1 - url[::-1].index('/')
  pii = url[ind+1:]
  print(pii)
  params['pii'] = pii
  # print(params)
  bibs_responses.append(requests.get(sd_url, headers=headers, params=params).text)
  # print(response.text)

S0950705117303106
S0263224118311692
S0092867416310078
B9780128213391000076
S1077314215000375
S152994301830247X
S0967064599001381
S092523121500675X
S0195666316308340
S0925231214003877
S0888754320311599
S0022098115300575
S095741741830558X
S2211124720300358
S0923596515001435
S0075951104800057
S0306457310001020
S0925231218302893
B978012814393300007X
S0034425719301166
S0920410519311027
S0167739X1630423X
S1074761318301845
S0022474X18300274
S1550413120304800
S0093691X18305818
S0092867418309012
S0957417417301604
S1534580716307523
S1470160X17305939
S0304394019307207
S0022098112001116
S1537511014000713
S0925231216305306
S1094919420300141
S037851731930506X
S1094996816300305
B9780128175767000031
S0006320718305718
S0341816219300578
S0004370215001034
S153561082030310X
S0092867420309946
S0034425718305534
S0093691X17301243
S0304401720300820
S0925231216303198
S0010945220303129
S2211124720306215
S0147651319314472
S0167701211003678
S0031320304000913
S1877959X17300213
S1875952116300258
S0169023X16303469
S

In [99]:
print(len(bibs_responses))

788


In [100]:
import pprint
bibs = []
for bib in bibs_responses[:1]:
  # print(bib)
  bibtex = bib.strip("}").strip("\t ") + '\n'
  # print(bibtex)
  ind = bibtex.index("{")
  bib_dict = {}
  bib_dict["type"] = bibtex[:ind]
  bibtex = bibtex[ind+1:]
  ind = bibtex.index("\n")
  bib_dict["id"] = bibtex[:ind-1]
  bibtex = bibtex[ind+1:]
  while(bibtex != "" and bibtex != "\n"): #len(bibtext) > 3):
    # print(bibtex)
    # print("bib", bibtex, len(bibtex))
    ind = bibtex.index("\n")
    attribute = bibtex[:ind].strip("\t,")
    ind_attr = attribute.index("=")
    bib_dict[attribute[:ind_attr].strip()] = attribute[ind_attr+1:].strip()
    bibtex = bibtex[ind + 1:]
  # pprint.p
  # print(bib_dict)
  bibs.append(bib_dict)

In [101]:
print(bibs)

[{'type': '@article', 'id': 'HAO201743', 'title': '"Leveraging question target word features through semantic relation expansion for answer type classification"', 'journal': '"Knowledge-Based Systems"', 'volume': '"133"', 'pages': '"43 - 52"', 'year': '"2017"', 'issn': '"0950-7051"', 'doi': '"https://doi.org/10.1016/j.knosys.2017.06.030"', 'url': '"http://www.sciencedirect.com/science/article/pii/S0950705117303106"', 'author': '"Tianyong Hao and Wenxiu Xie and Qingyao Wu and Heng Weng and Yingying Qu"', 'keywords': '"Answer type identification, Classification, Question target, WordNet"', 'abstract': '"Answer type classification is a vital step of question answering systems to detect the most suitable target answer type. Highly accurate identification and classification of an answer type can help identify users’ question targets and filter out irrelevant candidate answers to improve system performances. This paper proposes a novel hybrid approach, named as ATICM, for automated answer ty

In [102]:
output_json = {}
for bib in range(0, len(bibs)):
  print(bibs[bib])
  output_json[bib] = bibs[bib]

{'type': '@article', 'id': 'HAO201743', 'title': '"Leveraging question target word features through semantic relation expansion for answer type classification"', 'journal': '"Knowledge-Based Systems"', 'volume': '"133"', 'pages': '"43 - 52"', 'year': '"2017"', 'issn': '"0950-7051"', 'doi': '"https://doi.org/10.1016/j.knosys.2017.06.030"', 'url': '"http://www.sciencedirect.com/science/article/pii/S0950705117303106"', 'author': '"Tianyong Hao and Wenxiu Xie and Qingyao Wu and Heng Weng and Yingying Qu"', 'keywords': '"Answer type identification, Classification, Question target, WordNet"', 'abstract': '"Answer type classification is a vital step of question answering systems to detect the most suitable target answer type. Highly accurate identification and classification of an answer type can help identify users’ question targets and filter out irrelevant candidate answers to improve system performances. This paper proposes a novel hybrid approach, named as ATICM, for automated answer typ

In [103]:
print(output_json)

{0: {'type': '@article', 'id': 'HAO201743', 'title': '"Leveraging question target word features through semantic relation expansion for answer type classification"', 'journal': '"Knowledge-Based Systems"', 'volume': '"133"', 'pages': '"43 - 52"', 'year': '"2017"', 'issn': '"0950-7051"', 'doi': '"https://doi.org/10.1016/j.knosys.2017.06.030"', 'url': '"http://www.sciencedirect.com/science/article/pii/S0950705117303106"', 'author': '"Tianyong Hao and Wenxiu Xie and Qingyao Wu and Heng Weng and Yingying Qu"', 'keywords': '"Answer type identification, Classification, Question target, WordNet"', 'abstract': '"Answer type classification is a vital step of question answering systems to detect the most suitable target answer type. Highly accurate identification and classification of an answer type can help identify users’ question targets and filter out irrelevant candidate answers to improve system performances. This paper proposes a novel hybrid approach, named as ATICM, for automated answer

In [104]:
import json
filename = "/content/drive/My Drive/bibtex_sd.json"    
with open(filename, 'w') as outfile:
    json.dump(output_json, outfile)